# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

I. J. M. Crossfield  ->  I. J. M. Crossfield  |  ['I. J. M. Crossfield']
T. Henning  ->  T. Henning  |  ['T. Henning']
J. Eberhardt  ->  J. Eberhardt  |  ['J. Eberhardt']
J. Liu  ->  J. Liu  |  ['J. Liu']
N. Storm  ->  N. Storm  |  ['N. Storm']
M. Bergemann  ->  M. Bergemann  |  ['M. Bergemann']


G. Chauvin  ->  G. Chauvin  |  ['G. Chauvin']
T. Henning  ->  T. Henning  |  ['T. Henning']
L. Mohr  ->  L. Mohr  |  ['L. Mohr']
J. Liu  ->  J. Liu  |  ['J. Liu']
Arxiv has 104 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2412.05423


extracting tarball to tmp_2412.05423...

 done.
Retrieving document from  https://arxiv.org/e-print/2412.05609


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2412.05423/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'tables/PopSynthesisParams.tex' from 'tmp_2412.05423/tables/PopSynthesisParams.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure imadet_sharknir
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure imadet_sharknir as tmp_2412.06404/images/imadet_sharknir.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure imadet_lmircam
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure imadet_lmircam as tmp_2412.06404/images/imadet_lmircam.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))
/opt/hostedtoolcache/Python/3.9

Found 87 bibliographic references in tmp_2412.06404/HD57625.bbl.
Issues with the citations
syntax error in line 609: premature end of file
Retrieving document from  https://arxiv.org/e-print/2412.06412


extracting tarball to tmp_2412.06412...

 done.


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2412.06412/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'secs/4_discussion' from 'tmp_2412.06412/secs/4_discussion.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'secs/3_main_structure' from 'tmp_2412.06412/secs/3_main_structure.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.20/x64/

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.05609-b31b1b.svg)](https://arxiv.org/abs/2412.05609) | **TOI-4504: Exceptionally large Transit Timing Variations induced by two resonant warm gas giants in a three planet system**  |
|| M. Vítková, et al. -- incl., <mark>T. Henning</mark>, <mark>J. Eberhardt</mark> |
|*Appeared on*| *2024-12-10*|
|*Comments*| *20 pages, 12 figures, accepted in ApJL*|
|**Abstract**|            We present a joint analysis of TTVs and Doppler data for the transiting exoplanet system TOI-4504. TOI-4504 c is a warm Jupiter-mass planet that exhibits the largest known transit timing variations (TTVs), with a peak-to-node amplitude of $\sim$ 2 days, the largest value ever observed, and a super-period of $\sim$ 930 d. TOI-4504 b and c were identified in public TESS data, while the TTVs observed in TOI-4504 c, together with radial velocity (RV) data collected with FEROS, allowed us to uncover a third, non-transiting planet in this system, TOI-4504 d. We were able to detect transits of TOI-4504 b in the TESS data with a period of 2.4261$\pm 0.0001$ days and derive a radius of 2.69$\pm 0.19$ R$_{\oplus}$. The RV scatter of TOI-4504 was too large to constrain the mass of TOI-4504 b, but the RV signals of TOI-4504 c \& d were sufficiently large to measure their masses. The TTV+RV dynamical model we apply confirms TOI-4504 c as a warm Jupiter planet with an osculating period of 82.54$\pm 0.02$ d, mass of 3.77$\pm 0.18$ M$_{\rm J}$ and a radius of 0.99$\pm 0.05$ R$_{\rm J}$, while the non-transiting planet TOI-4504 d, has an orbital period of 40.56$\pm 0.04$ days and mass of 1.42$_{-0.06}^{+0.07}$ M$_{\rm J}$. We present the discovery of a system with three exoplanets: a hot sub-Neptune and two warm Jupiter planets. The gas giant pair is stable and likely locked in a first-order 2:1 mean-motion resonance (MMR). The TOI-4504 system is an important addition to MMR pairs, whose increasing occurrence supports a smooth migration into a resonant configuration during the protoplanetary disk phase.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.06277-b31b1b.svg)](https://arxiv.org/abs/2412.06277) | **NLTE abundances of Eu for a sample of metal-poor stars in the Galactic Halo and Metal-poor Disk with 1D and <3D> models**  |
|| Y. Guo, et al. -- incl., <mark>N. Storm</mark>, <mark>M. Bergemann</mark> |
|*Appeared on*| *2024-12-10*|
|*Comments*| **|
|**Abstract**|            Accurate measurements of europium abundances in cool stars are essential for an enhanced understanding of the r-process mechanisms. We measure the abundance of Eu in solar spectra and a sample of metal-poor stars in the Galactic halo and metal-poor disk, with the metallicities ranging from \GG{$-2.4$} to $-0.5$ dex, using non-local thermodynamic equilibrium (NLTE) line formation. We compare these measurements with Galactic Chemical Evolution (GCE) models to \GG{explore the impact of the NLTE corrections on the contribution of r-process site in Galactic chemical evolution. In this work, we use NLTE line formation, as well as one-dimensional (1D) hydrostatic and spatial averages of 3D hydrodynamical ($<$3D$>$) model atmospheres to measure the abundance of Eu based on both the Eu II 4129 Å and Eu II 6645 Å lines for solar spectra and metal-poor stars. We find that \GG{for Eu II 4129 Å line the NLTE modelling leads to higher (0.04 dex) solar Eu abundance in 1D and higher (0.07 dex) in \GG{$<$3D$>$} NLTE while} NLTE modelling leads to higher (0.01 dex) solar Eu abundance in 1D and lower (0.03 dex) in \GG{$<$3D$>$} NLTE for Eu II 6645 Å line. Although the NLTE corrections for the Eu II $\lambda$ 4129 Å and Eu II $\lambda$ 6645 Å lines are opposite, the discrepancy between the abundances derived from these individual lines reduces after applying NLTE corrections, highlighting the critical role of NLTE abundance determinations. By comparing these measurements with Galactic chemical evolution (GCE) models, we find that the \G{amount of NLTE correction does not require significant change of the parameters for Eu production} in the GCE models.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.06404-b31b1b.svg)](https://arxiv.org/abs/2412.06404) | **A multi-technique detection of an eccentric giant planet around accelerating star HD 57625**  |
|| D. Barbato, et al. -- incl., <mark>G. Chauvin</mark>, <mark>T. Henning</mark>, <mark>L. Mohr</mark> |
|*Appeared on*| *2024-12-10*|
|*Comments*| *12 pages, 8 figures, accepted for publication in A&A*|
|**Abstract**|            The synergy between different detection methods is a key asset in exoplanetology, allowing for both precise characterization of detected exoplanets and robust constraints even in the case of non-detection. Recently, the interplay between imaging, radial velocities and astrometry has produced significant advancements in exoplanetary science. We report a first result of an ongoing survey performed with SHARK-NIR, the new high-contrast near-infrared imaging camera at the Large Binocular Telescope, in parallel with LBTI/LMIRCam in order to detect planetary companions around stars with significant proper motion anomaly. In this work we focus on HD 57625, a F8 star for which we determine a $4.8^{+3.7}_{-2.9}$Ga age, exhibiting significant astrometric acceleration and for which archival radial velocities hint at the presence of a previously undetected massive long-period companion. We analyse the imaging data we collected with SHARK-NIR and LMIRCam in synergy with the available public SOPHIE radial velocity time series and Hipparcos-Gaia proper motion anomaly. With this joint multi-technique analysis, we aim at characterizing the companion responsible for the astrometric and radial velocity signals. The imaging observations result in a non-detection, indicating the companion to be in the substellar regime. This is confirmed by the synergic analysis of archival radial velocity and astrometric measurements resulting in the detection of HD 57625 b, a ${8.43}_{-0.91}^{+1.10}$M$_{\rm Jup}$ planetary companion with an orbital separation of ${5.70}_{-0.13}^{+0.14}$au and ${0.52}_{-0.03}^{+0.04}$ eccentricity. HD 57625 b joins the small but growing population of giant planets in outer orbits with true mass determination provided by the synergic usage of multiple detection methods, proving once again the importance of multi-technique analysis in providing robust characterization of planetary companions.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.05423-b31b1b.svg)](https://arxiv.org/abs/2412.05423) | **A joint effort to discover and characterize two resonant mini Neptunes around TOI-1803 with TESS, HARPS-N and CHEOPS**  |
|| T. Zingales, et al. -- incl., <mark>I. J. M. Crossfield</mark> |
|*Appeared on*| *2024-12-10*|
|*Comments*| *26 Pages, 21 Figures Accepted for Publication in Astronomy & Astrophysics*|
|**Abstract**|            We present the discovery of two mini Neptunes near a 2:1 orbital resonance configuration orbiting the K0 star TOI-1803. We describe their orbital architecture in detail and suggest some possible formation and evolution scenarios. Using CHEOPS, TESS, and HARPS-N datasets we can estimate the radius and the mass of both planets. We used a multidimensional Gaussian Process with a quasi-periodic kernel to disentangle the planetary components from the stellar activity in the HARPS-N dataset. We performed dynamical modeling to explain the orbital configuration and performed planetary formation and evolution simulations. For the least dense planet, we define possible atmospheric characterization scenarios with simulated JWST observations. TOI-1803 b and TOI-1803 c have orbital periods of $\sim$6.3 and $\sim$12.9 days, respectively, residing in close proximity to a 2:1 orbital resonance. Ground-based photometric follow-up observations revealed significant transit timing variations (TTV) with an amplitude of $\sim$10 min and $\sim$40 min, respectively, for planet -b and -c. With the masses computed from the radial velocities data set, we obtained a density of (0.39$\pm$0.10) $\rho_{earth}$ and (0.076$\pm$0.038) $\rho_{earth}$ for planet -b and -c, respectively. TOI-1803 c is among the least dense mini Neptunes currently known, and due to its inflated atmosphere, it is a suitable target for transmission spectroscopy with JWST. We report the discovery of two mini Neptunes close to a 2:1 orbital resonance. The detection of significant TTVs from ground-based photometry opens scenarios for a more precise mass determination. TOI-1803 c is one of the least dense mini Neptune known so far, and it is of great interest among the scientific community since it could constrain our formation scenarios.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.06175-b31b1b.svg)](https://arxiv.org/abs/2412.06175) | **Classification of Periodic Variable Stars from TESS**  |
|| X. Gao, X. Chen, S. Wang, <mark>J. Liu</mark> |
|*Appeared on*| *2024-12-10*|
|*Comments*| *29 pages, 18 figures, accepted for publication in ApJS*|
|**Abstract**|            The number of known periodic variable stars has increased rapidly in recent years. As an all-sky transit survey, the Transiting Exoplanet Survey Satellite (TESS) plays an important role in detecting low-amplitude variable stars. Using 2-minute cadence data from the first 67 sectors of TESS, we find 72,505 periodic variable stars. We used 19 parameters including period, physical parameters, and light curve (LC) parameters to classify periodic variable stars into 12 sub-types using random forest method. Pulsating variable stars and eclipsing binaries are distinguished mainly by period, LC parameters and physical parameters. GCAS, ROT, UV, YSO are distinguished mainly by period and physical parameters. Compared to previously published catalogs, 63,106 periodic variable stars (87.0$\%$) are newly classified, including 13 Cepheids, 27 RR Lyrae stars, $\sim$4,600 $\delta$ Scuti variable stars, $\sim$1,600 eclipsing binaries, $\sim$34,000 rotational variable stars, and about 23,000 other types of variable stars. The purity of eclipsing binaries and pulsation variable stars ranges from 94.2$\%$ to 99.4$\%$ when compared to variable star catalogs of Gaia DR3 and ZTF DR2. The purity of ROT is relatively low at 83.3$\%$. The increasing number of variables stars is helpful to investigate the structure of the Milky Way, stellar physics, and chromospheric activity.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.06412-b31b1b.svg)](https://arxiv.org/abs/2412.06412) | **StarWhisper Telescope: Agent-Based Observation Assistant System to Approach AI Astrophysicist**  |
|| C. Wang, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2024-12-10*|
|*Comments*| *21 pages, 18 figures*|
|**Abstract**|            With the rapid advancements in Large Language Models (LLMs), LLM-based agents have introduced convenient and user-friendly methods for leveraging tools across various domains. In the field of astronomical observation, the construction of new telescopes has significantly increased astronomers' workload. Deploying LLM-powered agents can effectively alleviate this burden and reduce the costs associated with training personnel. Within the Nearby Galaxy Supernovae Survey (NGSS) project, which encompasses eight telescopes across three observation sites, aiming to find the transients from the galaxies in 50 mpc, we have developed the \textbf{StarWhisper Telescope System} to manage the entire observation process. This system automates tasks such as generating observation lists, conducting observations, analyzing data, and providing feedback to the observer. Observation lists are customized for different sites and strategies to ensure comprehensive coverage of celestial objects. After manual verification, these lists are uploaded to the telescopes via the agents in the system, which initiates observations upon neutral language. The observed images are analyzed in real-time, and the transients are promptly communicated to the observer. The agent modifies them into a real-time follow-up observation proposal and send to the Xinglong observatory group chat, then add them to the next-day observation lists. Additionally, the integration of AI agents within the system provides online accessibility, saving astronomers' time and encouraging greater participation from amateur astronomers in the NGSS project.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2412.05609/./GLS_TOI-4504_FEROS_in_Per_2_2.png', 'tmp_2412.05609/./holczer.png', 'tmp_2412.05609/./TPF_Gaia_TIC349972412_S28.png']
copying  tmp_2412.05609/./GLS_TOI-4504_FEROS_in_Per_2_2.png to _build/html/
copying  tmp_2412.05609/./holczer.png to _build/html/
copying  tmp_2412.05609/./TPF_Gaia_TIC349972412_S28.png to _build/html/
exported in  _build/html/2412.05609.md
    + _build/html/tmp_2412.05609/./GLS_TOI-4504_FEROS_in_Per_2_2.png
    + _build/html/tmp_2412.05609/./holczer.png
    + _build/html/tmp_2412.05609/./TPF_Gaia_TIC349972412_S28.png
found figures ['tmp_2412.06277/figures/./Eubanchmark_sun_spectrum.png', 'tmp_2412.06277/figures/./NLTE_results.png', 'tmp_2412.06277/figures/./EuII_grotrian_diagram4129.png']
copying  tmp_2412.06277/figures/./Eubanchmark_sun_spectrum.png to _build/html/
copying  tmp_2412.06277/figures/./NLTE_results.png to _build/html/
copying  tmp_2412.06277/figures/./EuII_grotrian_diagram4129.png to _build/html/
exported in  _build/html/2

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\autoref}$
$\newcommand{\thetable}{A\arabic{table}}$
$\newcommand{\thefigure}{A\arabic{figure}}$
$\newcommand{\equationautorefname}{Eq.}$
$\newcommand{\figureautorefname}{Fig.}$
$\newcommand{\sectionautorefname}{Sect.}$
$\newcommand{\subsectionautorefname}{Sect.}$
$\newcommand{\subsubsectionautorefname}{Sect.}$</div>



<div id="title">

# TOI-4504: Exceptionallylarge Transit Timing Variations \ induced by two resonant warm gas giants in a three planet system.

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2412.05609-b31b1b.svg)](https://arxiv.org/abs/2412.05609)<mark>Appeared on: 2024-12-10</mark> -  _20 pages, 12 figures, accepted in ApJL_

</div>
<div id="authors">

M. Vítková, et al. -- incl., <mark>T. Henning</mark>, <mark>J. Eberhardt</mark>

</div>
<div id="abstract">

**Abstract:** We present a joint analysis of TTVs and Doppler data for the transiting exoplanet system TOI-4504. TOI-4504 c is a warm Jupiter-mass planet that exhibits the largest known transit timing variations (TTVs), with a peak-to-node amplitude of $\sim$ 2 days, the largest value ever observed, and a super-period of $\sim$ 930 d. TOI-4504 b and c were identified in public TESS data, while the TTVs observed in TOI-4504 c, together with radial velocity (RV) data collected with FEROS, allowed us to uncover a third, non-transiting planet in this system, TOI-4504 d. We were able to detect transits of TOI-4504 b in the TESS data with a period of 2.4261 $\pm 0.0001$ days and derive a radius of 2.69 $\pm 0.19$ R $_{\oplus}$ . The RV scatter of TOI-4504 was too large to constrain the mass of TOI-4504 b, but the RV signals of TOI-4504 c \& d were sufficiently large to measure their masses. The TTV+RV dynamical model we apply confirms TOI-4504 c as a warm Jupiter planet with an osculating period of 82.54 $\pm 0.02$ d, mass of 3.77 $\pm 0.18$ M $_{\rm J}$ and a radius of 0.99 $\pm 0.05$ R $_{\rm J}$ , while the non-transiting planet TOI-4504 d, has an orbital period of 40.56 $\pm 0.04$ days and mass of 1.42 $_{-0.06}^{+0.07}$ M $_{\rm J}$ . We present the discovery of a system with three exoplanets: a hot sub-Neptune and two warm Jupiter planets. The gas giant pair is stable and likely locked in a first-order 2:1 mean-motion resonance (MMR). The TOI-4504 system is an important addition to MMR pairs, whose increasing occurrence supports a smooth migration into a resonant configuration during the protoplanetary disk phase.

</div>

<div id="div_fig1">

<img src="tmp_2412.05609/./GLS_TOI-4504_FEROS_in_Per_2_2.png" alt="Fig5" width="100%"/>

**Figure 5. -** GLS power spectrum of FEROS spectroscopic products of TOI-4504. From top to bottom panels, as labeled, RVs used in this work, RV residuals after subtracting the dominant signal of TOI-4504 c at 84 d, the final the best-fit TTV+RV model residuals, BIS, $\rm{H}_{\alpha}$, He I  Na II, and $\log(R'_{HK})$ activity indicators, respectively. False alarm probability levels of 10\%, 1\%, and 0.1\% are marked with dashed lines, respectively. The red and magenta vertical lines indicate the best-fit periods of the transiting Jovian planet TOI-4504 c, and the non-transiting TOI-4504 d, respectively.  (*activity*)

</div>
<div id="div_fig2">

<img src="tmp_2412.05609/./holczer.png" alt="Fig8" width="100%"/>

**Figure 8. -** Position of TOI-4504 c, other planets with significant TTVs and planets from [Holczer, Mazeh and Nachmani (2016)]() in period-TTV amplitude and period-TTV amplitude and period ratio diagram. TTV amplitude is a peak-to-node amplitude of cosinus fit, for Kepler-90 g it is the maximum observed difference between the time lapsed between consecutive transits. (*holczer*)

</div>
<div id="div_fig3">

<img src="tmp_2412.05609/./TPF_Gaia_TIC349972412_S28.png" alt="Fig1" width="100%"/>

**Figure 1. -** Target Pixel Files in Sector 28 for TOI-4504 obtained with {$\textsc$ tpfplotter}. The shape of the aperture mask used to extract the photometry is marked with orange. Red dots indicate the sources of the Gaia DR3 catalogue in the field. TOI-4504 is marked with a white cross. (*TPF*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2412.05609"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\GG}[1]{\textcolor{black}{#1}}$
$\newcommand{\G}[1]{\textcolor{black}{#1}}$
$\newcommand{\jl}[1]{\textcolor{orange}{#1}}$</div>



<div id="title">

# NLTE abundances of Eu for a sample of metal-poor stars in the Galactic Halo and Metal-poor Disk with 1D and $<$3D$>$ models

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2412.06277-b31b1b.svg)](https://arxiv.org/abs/2412.06277)<mark>Appeared on: 2024-12-10</mark> - 

</div>
<div id="authors">

Y. Guo, et al. -- incl., <mark>N. Storm</mark>, <mark>M. Bergemann</mark>

</div>
<div id="abstract">

**Abstract:** As a key to chemical evolutionary studies, the distribution of elements in galactic provides a wealth of information to understand the individual star formation histories of galaxies.The r-process is a complex nucleosynthesis process, and the origin of r-process elements is heavily debated.Europium (Eu) is viewed as an almost pure r-process element.Accurate measurements of europium abundances in cool stars are essential for an enhanced understanding of the r-process mechanisms. We measure the abundance of Eu in solar spectra and a sample of metal-poor stars in the Galactic halo and metal-poor disk, with the metallicities ranging from $\GG{$-2.4$}$ to $-0.5$ dex, using non-local thermodynamic equilibrium (NLTE) line formation. We compare these measurements with Galactic Chemical Evolution (GCE) models to $\GG{explore the impact of the NLTE corrections on the contribution of r-process site in Galactic chemical evolution.}$ In this work, we use NLTE line formation, as well as one-dimensional (1D) hydrostatic and spatial averages of 3D hydrodynamical ( $<$ 3D $>$ ) model atmospheres to measure the abundance of Eu based on both the Eu II 4129 Å and Eu II 6645 Å lines for solar spectra and metal-poor stars. We find that $\GG{for Eu II 4129 Å line the NLTE modelling leads to higher (0.04 dex) solar Eu abundance in 1D and higher (0.07 dex) in \GG{$<$3D$>$} NLTE while}$ NLTE modelling leads to higher (0.01 dex) solar Eu abundance in 1D and lower (0.03 dex) in $\GG{$<$3D$>$}$ NLTE for Eu II 6645 Å line. $\GG{Although the NLTE corrections for the Eu II $\lambda$ 4129 Å and Eu II $\lambda$ 6645 Å lines are opposite, the discrepancy between the abundances derived from these individual lines reduces after applying NLTE corrections, highlighting the critical role of NLTE abundance determinations.}$ By comparing these measurements with Galactic chemical evolution (GCE) models, we find that the $\G{amount of NLTE correction does not require significant change of the parameters for Eu production}$ in the GCE models.

</div>

<div id="div_fig1">

<img src="tmp_2412.06277/figures/./Eubanchmark_sun_spectrum.png" alt="Fig3" width="100%"/>

**Figure 3. -** The best fitting synthetic spectra of sun for $\GG${$\lambda$ 4129Å (left panels)} and Eu II $\lambda$ 6645Å (right panels) generated from TSFitPy. The red line and blue line represent the synthetic spectra based on 1D and $<$3D$>$ NLTE, respectively. The observed data are shown as black dots. (*fig: Eufit_sun_spectrum*)

</div>
<div id="div_fig2">

<img src="tmp_2412.06277/figures/./NLTE_results.png" alt="Fig8" width="100%"/>

**Figure 8. -** The trend of [Eu/Fe] based on the best fitting results of 1D NLTE and LTE.
    $\GG${The green squares and blue dots represent the averaged NLTE and LTE [Eu/Fe] ratios across the selected bins.
    The yellow triangles and cyan circles represent the NLTE and LTE [Eu/Fe] for each stars.
    The purple dots represent 1D LTE Eu abundance measurements of 1274 metal-rich stars from Gaia-ESO survey.
    }
    The dashed and solid lines represent the GCE models with different fraction of massive stars that end up with MRSN instead of CCSN.  (*fig: Eu_trend*)

</div>
<div id="div_fig3">

<img src="tmp_2412.06277/figures/./EuII_grotrian_diagram4129.png" alt="Fig1" width="100%"/>

**Figure 1. -** Grotrian diagram of the $\GG${Eu II model atom}. The model atom are taken from the $\GG${\cite{2024A&AStorm}}. The $\GG${blue} and red lines represent the transitions giving rise to the $\GG${Eu II 4129 Å } and 6645 Å lines, respectively. (*fig: Grotrian diagrams*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2412.06277"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\gcm}{\ensuremath{ \rm{g} \rm{cm}^{-3}}\xspace}$
$\newcommand{\yr}{\ensuremath{ \rm a}\xspace}$
$\newcommand{\pc}{\ensuremath{ \rm pc}\xspace}$
$\newcommand{\Lsun}{\ensuremath{ {\rm L}_{\odot}}\xspace}$
$\newcommand{\Msun}{\ensuremath{ {\rm M}_{\odot}}\xspace}$
$\newcommand{\Rsun}{\ensuremath{ {\rm R}_{\odot}}\xspace}$
$\newcommand{\Mjup}{\ensuremath{ {\rm M}_{\rm Jup}}\xspace}$
$\newcommand{\mps}{\ensuremath{ {\rm m s}^{-1}}\xspace}$
$\newcommand{\kmps}{\ensuremath{ {\rm km s}^{-1}}\xspace}$
$\newcommand{\days}{\ensuremath{ \rm d}\xspace}$
$\newcommand{\au}{\ensuremath{ \rm au}\xspace}$
$\newcommand{\deg}{\ensuremath{ \rm deg}\xspace}$
$\newcommand{\sresinob}{\ensuremath{\sqrt{e_{\rm b}}\sin{\omega_{\rm b}}}\xspace}$
$\newcommand{\srecosob}{\ensuremath{\sqrt{e_{\rm b}}\cos{\omega_{\rm b}}}\xspace}$
$\newcommand$
$\newcommand$
$\newcommand{\gaia}{Gaia\xspace}$
$\newcommand{\hip}{Hipparcos\xspace}$
$\newcommand{\orvara}{\texttt{orvara}\xspace}$
$\newcommand{\exofast}{\texttt{EXOFASTv2}\xspace}$
$\newcommand{\smw}{\ensuremath{S_{\rm MW}}\xspace}$
$\newcommand{\deg}{\ensuremath{ {\rm deg}}\xspace}$
$\newcommand{\fd}{\ensuremath{^{\mathrm d}}}$
$\newcommand{\fh}{\ensuremath{^{\mathrm h}}}$
$\newcommand{\fm}{\ensuremath{^{\mathrm m}}}$
$\newcommand{\fs}{\ensuremath{^{\mathrm s}}}$
$\newcommand{\fdg}{\ensuremath{^\circ}}$
$\newcommand{\fmin}{\ensuremath{^\prime}}$
$\newcommand{\fsec}{\ensuremath{^{\prime\prime}}}$
$\newcommand{\teff}{\ensuremath{T_{\rm eff}}\xspace}$
$\newcommand{\logg}{\ensuremath{\log g}\xspace}$
$\newcommand{\gmag}{\ensuremath{G}\xspace}$
$\newcommand{\gbp}{\ensuremath{G_{\rm BP}}\xspace}$
$\newcommand{\grp}{\ensuremath{G_{\rm RP}}\xspace}$
$\newcommand{\parallax}{\ensuremath{\varpi}\xspace}$
$\newcommand{\feh}{\ensuremath{[\rm Fe/H]}\xspace}$
$\newcommand{\uprior}{\ensuremath{\mathcal{U}}}$
$\newcommand{\mum}{{ \mum}\xspace}$
$\newcommand{\mas}{ {mas}\xspace}$
$\newcommand{\masyr}{{mas a^{-1}}\xspace}$</div>



<div id="title">

# A multi-technique detection of an eccentric giant planet around accelerating star HD 57625

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2412.06404-b31b1b.svg)](https://arxiv.org/abs/2412.06404)<mark>Appeared on: 2024-12-10</mark> -  _12 pages, 8 figures, accepted for publication in A&A_

</div>
<div id="authors">

D. Barbato, et al. -- incl., <mark>G. Chauvin</mark>, <mark>T. Henning</mark>, <mark>L. Mohr</mark>

</div>
<div id="abstract">

**Abstract:** The synergy between different detection methods is a key asset in exoplanetology, allowing for both precise characterization of detected exoplanets and robust constraints even in the case of non-detection. Recently, the interplay between imaging, radial velocities and astrometry has produced significant advancements in exoplanetary science. We report a first result of an ongoing survey performed with SHARK-NIR, the new high-contrast near-infrared imaging camera at the Large Binocular Telescope, in parallel with LBTI/LMIRCam in order to detect planetary companions around stars with significant proper motion anomaly. In this work we focus on HD 57625, a F8 star for which we determine a $4.8^{+3.7}_{-2.9}$ Ga age, exhibiting significant astrometric acceleration and for which archival radial velocities hint at the presence of a previously undetected massive long-period companion. We analyse the imaging data we collected with SHARK-NIR and LMIRCam in synergy with the available public SOPHIE radial velocity time series and Hipparcos-Gaia proper motion anomaly. With this joint multi-technique analysis, we aim at characterizing the companion responsible for the astrometric and radial velocity signals. The imaging observations result in a non-detection, indicating the companion to be in the substellar regime. This is confirmed by the synergic analysis of archival radial velocity and astrometric measurements resulting in the detection of HD 57625 b, a ${8.43}_{-0.91}^{+1.1}$ $\Mjup$ planetary companion with an orbital separation of ${5.70}_{-0.13}^{+0.14}$ $\au$ and ${0.52}_{-0.03}^{+0.04}$ eccentricity. HD 57625 b joins the small but growing population of giant planets in outer orbits with true mass determination provided by the synergic usage of multiple detection methods, proving once again the importance of multi-technique analysis in providing robust characterization of planetary companions.

</div>

<div id="div_fig1">

<img src="tmp_2412.06404/images/pma.png" alt="Fig4" width="100%"/>

**Figure 4. -** Proper motion anomaly sensitivity curve for HD 57625. The dark green curve shows the PMa-compatible companion masses as a function of orbital separations, the shaded region corresponding to the 1$\sigma$ uncertainty range. The blue and cyan curves represent the SHARK-NIR and LMIRCam mass limits as in Fig. \ref{fig:contrast}. Horizontal lines indicate the  brown dwarf (brown) and stellar mass (orange) thresholds. The positions of HD 57625 b as obtained by the RV-only fit and by the joint RV and PMa fit are shown as a red and white circle, respectively, while the distant stellar companion is shown as a yellow star. (*fig:pma*)

</div>
<div id="div_fig2">

<img src="tmp_2412.06404/images/contrast.png" alt="Fig1" width="100%"/>

**Figure 1. -** _Top panel:_ contrast curve for the imaging observation of HD 57625 conducted on the night of February 24th 2024 UT. _Bottom panel_: mass limits derived using the AMES-COND models, the thick curve corresponding to the nominal 4.8 Ga stellar age, the shaded region corresponding to the age uncertainty. In both panels, the blue solid curves refer to the SHARK-NIR H-band observations and the cyan dashed lines refer to the LMIRCam L$^\prime$-band observation, with similarly colour-coded vertical dotted lines indicating the Inner Working Angle (IWA) of each instrument. (*fig:contrast*)

</div>
<div id="div_fig3">

<img src="tmp_2412.06404/images/rvgls.png" alt="Fig2" width="100%"/>

**Figure 2. -** _Left panels_: time series for the SOPHIE (orange triangles) and SOPHIE+ (red circles) radial velocity data of HD 57625, residual time series obtained after removing the detected Keplerian signal and activity indexes. For the activity indexes time series, the Pearson correlation coefficient with both original and residual RVs are noted. _Right panels_: corresponding generalised Lomb-Scargle periodograms of the time series, with horizontal solid, dashed and dotted lines marking the 10\%, 1\% and 0.1\% FAP thresholds. (*fig:rvgls*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2412.06404"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

274  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

16  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
